In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from bs4 import BeautifulSoup
import requests

C:\Users\Client\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#to read html pages from website
web_data=requests.get("https://www.mygov.in/corona-data/covid19-statewise-status")
soup = BeautifulSoup(web_data.text, "html.parser")

In [3]:
link_head=soup.find('div',class_="field field-name-field-district-reporting field-type-text field-label-above")
district_link=link_head.find('div',class_="field-item even").text

In [4]:
content=soup.find_all('div',class_="content")

In [5]:
#fetching required information from html tag and adding to csv file
filename="covid.csv"
f=open(filename, "w", encoding="utf-8")
f.write("{},{},{},{}\n".format("State_Name","Total_Confirmed","Total_Cured","Total_Death"))
for con in content[3:]:
    if con.find('div',class_="field field-name-field-select-state field-type-list-text field-label-above"):
        statename_field=con.find('div',class_="field field-name-field-select-state field-type-list-text field-label-above")
        if statename_field.find("div", class_= "field-item even"):
            state_name=statename_field.find('div',class_="field-item even").text
    if con.find('div',class_="field field-name-field-total-confirmed-indians field-type-number-integer field-label-above"):
        Total_confirmed=con.find('div',class_="field field-name-field-total-confirmed-indians field-type-number-integer field-label-above")
        if Total_confirmed.find("div", class_= "field-item even"):
            Total_confirmed=Total_confirmed.find('div',class_="field-item even").text
    if con.find('div',class_="field field-name-field-cured field-type-number-integer field-label-above"):
        Total_cured=con.find('div',class_="field field-name-field-cured field-type-number-integer field-label-above")
        if Total_cured.find("div", class_= "field-item even"):
            Total_cured=Total_cured.find('div',class_="field-item even").text
    if con.find('div',class_="field field-name-field-deaths field-type-number-integer field-label-above"):
        Total_death=con.find('div',class_="field field-name-field-deaths field-type-number-integer field-label-above")
        if Total_death.find("div", class_= "field-item even"):
            Total_death=Total_death.find('div',class_="field-item even").text
    csv=("{},{},{},{}\n".format(state_name,Total_confirmed,Total_cured,Total_death))
    f.write(csv)
f.close()    

In [6]:
data=pd.read_csv("covid.csv")
data.head()

,State_Name,Total_Confirmed,Total_Cured,Total_Death
0,AndhraPradesh,381,11,6
1,AndamanNicobar,11,10,0
2,Bihar,64,19,1
3,Chandigarh,19,7,0
4,Chhattisgarh,25,10,0


In [7]:
#to read pdf data to get district information
import tabula
district_data=tabula.read_pdf(district_link,pages='all')

In [8]:
district_data.columns=['State','No_of_Dist_Affected','District','Positive_cases']
district_data=district_data[1:]

In [9]:
district_data.head()

,State,No_of_Dist_Affected,District,Positive_cases
1,ANDAMAN AND NICOBAR ISLAND,1,ANDAMAN,10
2,ANDHRA PRADESH,11*,ANANTAPUR,2
3,NaN,NaN,CHITTOOR,10
4,NaN,NaN,EAST GODAVARI,11
5,NaN,NaN,GUNTUR,30


In [10]:
#we will drop "No_of_Dist_Affected",since we can get this value by groupby fumction if required
district_data.drop('No_of_Dist_Affected',axis=1,inplace=True)

In [11]:
#we are dropping rows having "District" or "Positive_cases" as nan , since we will not get any information from those rows
district_data.dropna(subset=['District','Positive_cases'],axis=0,inplace=True)
district_data.shape

(241, 3)

In [12]:
#fill the missing state value will forword fill, since pdf has one to many, 
#state to district rows these missing value has generated in data frame.
district_data['State']=district_data['State'].fillna(method='ffill')

In [13]:
district_data=district_data.reset_index()
district_data.drop('index',axis=1,inplace=True)
district_data.head()

,State,District,Positive_cases
0,ANDAMAN AND NICOBAR ISLAND,ANDAMAN,10
1,ANDHRA PRADESH,ANANTAPUR,2
2,ANDHRA PRADESH,CHITTOOR,10
3,ANDHRA PRADESH,EAST GODAVARI,11
4,ANDHRA PRADESH,GUNTUR,30


In [14]:
district_data.to_csv("district_data.csv")